In [1]:
import PyPDF2
import os

In [2]:
def get_text(path):
    reader = PyPDF2.PdfReader(path)
    text=""
    for page in reader.pages:
        text += page.extract_text().lower()
    return text

In [3]:
whole_text = get_text("attention_paper.pdf")
path = os.path.join("pdf_save", "1607.06450.pdf")
whole_text = get_text(path)
whole_text = whole_text.split("references")[0]

In [4]:
# find max length of blurb without newline
max_len = 0
for line in whole_text.split("\n"):
    if len(line) > max_len:
        max_len = len(line)

print(max_len)

240


In [5]:
# split when there is a newline and the line is less than half of the max length
blurb_list = []
blurb = ""
for line in whole_text.split("\n"):
    if len(line) < max_len/3:
        if blurb != "":
            blurb += line.lower()
            blurb_list.append(blurb)
        blurb = ""
    else:
        # lowercase and remove punctuation
        line = line.lower()
        blurb += line + " "


In [6]:
for blurb in blurb_list:
    print(blurb)
    print("")

training state-of-the-art, deep neural networks is computationally expensive. one way to reduce the training time is to normalize the activities of the neurons. a recently introduced technique called batch normalization uses the distribution of the summed input to a neuron over a mini-batch of training cases to compute a

neuron on each training case. this signiﬁcantly reduces the training time in feed- forward neural networks. however, the effect of batch normalization is dependent on the mini-batch size and it is not obvious how to apply it to recurrent neural net- works. in this paper, we transpose batch normalization into layer normalization by computing the mean and variance used for normalization from all of the summed

inputs to the neurons in a layer on a single training case. like batch normalization, we also give each neuron its own adaptive bias and gain which are applied after

the normalization but before the non-linearity. unlike batch normalization, layer normalization p

In [7]:
print(len(blurb_list))

63


In [8]:
from rank_bm25 import BM25Okapi
tokenized_corpus = [doc.split(" ") for doc in blurb_list]
bm25 = BM25Okapi(tokenized_corpus)


In [9]:
def bm25_search(query, corpus):
    tokenized_corpus = [doc.split(" ") for doc in corpus]
    bm25 = BM25Okapi(tokenized_corpus)
    query = query.lower()
    tokenized_query = query.split(" ")
    doc_scores = bm25.get_scores(tokenized_query)
    # get the index of the highest score

    ranked_docs = [corpus[i] for i in sorted(range(len(doc_scores)), key=lambda i: doc_scores[i], reverse=True)]
    return ranked_docs[0]

In [10]:
query = "we employ a residual connection around each of the two sub-layesr, followed by layer normalization."
print(bm25_search(query, blurb_list))

the proposed layer normalization is related to batch normalization and weight normalization. al- though, their normalization scalars are computed differently, these methods can be summarized as normalizing the summed inputs aito a neuron through the two scalars and. they also learn an adaptive bias band gaingfor each neuron after the normalization.


In [11]:
from sentence_transformers import CrossEncoder, util
def crossencoder_response(sentences, query):
    cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
    cross_inp = [[query, sent] for sent in sentences]
    cross_scores = cross_encoder.predict(cross_inp)
    scores_and_sents = list(zip(cross_scores, sentences))
    scores_and_sents.sort(reverse=True)
    return scores_and_sents[0][1]

In [12]:
print(crossencoder_response(blurb_list, query))

we have also experimented with convolutional neural networks. in our preliminary experiments, we observed that layer normalization offers a speedup over the baseline model without normalization, but batch normalization outperforms the other methods. with fully connected layers, all the hidden units in a layer tend to make similar contributions to the ﬁnal prediction and re-centering and re- scaling the summed inputs to a layer works well. however, the assumption of similar contributions is no longer true for convolutional neural networks. the large number of the hidden units whose receptive ﬁelds lie near the boundary of the image are rarely turned on and thus have very different statistics from the rest of the hidden units within the same layer. we think further research is needed to make layer normalization work well in convnets.


In [3]:
from reference_sentences import retrieve_reference_dicts
from arxiv_num_extraction import get_arxiv_numbers_from_pdf
reference_dicts = retrieve_reference_dicts('attention_paper.pdf')
arxiv_nums = get_arxiv_numbers_from_pdf('attention_paper.pdf')




['1607.06450',
 '1409.0473',
 '1703.03906',
 '1601.06733',
 '1406.1078',
 '1610.02357',
 '1412.3555',
 None,
 '1705.03122v2',
 '1308.0850',
 None,
 None,
 None,
 None,
 '1602.02410',
 None,
 None,
 '1610.10099v2',
 None,
 None,
 '1703.10722',
 '1703.03130',
 '1511.06114',
 '1508.04025',
 None,
 None,
 None,
 '1705.04304',
 None,
 '1608.05859',
 '1508.07909',
 '1701.06538',
 None,
 None,
 None,
 '1512.00567',
 None,
 '1609.08144',
 '1606.04199',
 None]